# Monadic Parsing

Based on Graham Huttons's "Programming in Haskell". This notebook showcases how monads
can be used to implement parsers.

"A parser is a program that takes a string of characters as input, and produces
some form of tree that makes the syntactic structure of the string explicit." (Graham Hutton)

In [1]:
using Pkg
Pkg.activate(".")
import FunctionWrappers: FunctionWrapper

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/CategoryTheory`
[ Info: Precompiling FunctionWrappers [069b7b12-0de2-55c6-9aab-29f3d0a68a2e]


In [2]:
struct Parser{a}
    p::FunctionWrapper{Vector{Tuple{a, String}},Tuple{String}}
end


f = FunctionWrapper{Vector{Tuple{Int, String}},Tuple{String}}(x::String -> [(1,x)]);
P{T} = FunctionWrapper{Vector{Tuple{T, String}},Tuple{String}}
typeof(f)

Parser(f)

Parser{Int64}(FunctionWrapper{Vector{Tuple{Int64, String}}, Tuple{String}}(Ptr{Nothing} @0x00007fee7e736190, Ptr{Nothing} @0x00007fee8b6f8010, Base.RefValue{var"#5#6"}(var"#5#6"()), var"#5#6"))

In [3]:
parse(p::Parser{T}, s::String) where T = p.p(s)
p = Parser(f)
parse(p,"ok")

1-element Vector{Tuple{Int64, String}}:
 (1, "ok")

In [4]:
# item::Parser{Char}
item = Parser(P{Char}(
        (inp -> inp == "" ? [] : [(inp[1], string(inp[2:end]))])
        ))

parse(item, "")
parse(item, "abc")

1-element Vector{Tuple{Char, String}}:
 ('a', "bc")

In [5]:
fmap(f::Function, p::Parser{T}) where T = Parser(P{T}(
        inp -> begin
            res = parse(p, inp)
            res == [] ? [] : [(f(res[1][1]),res[1][2])]
        end
    ))


fmap (generic function with 1 method)

In [6]:
parse(fmap(uppercase, item),"")
parse(fmap(uppercase, item),"abc")

1-element Vector{Tuple{Char, String}}:
 ('A', "bc")

In [161]:
η(v::T) where T = Parser(P{T}(inp -> [(v,inp)]))
bind(p::Parser{a},f::FunctionWrapper{Parser{b}, Tuple{a}}) where {a,b} = Parser(P{b}(
        inp -> begin
            res = parse(p, inp)
            @show res
            res == [] ? [] : parse(f(res[1][1]), res[1][2])
        end
    ))

η(1).p("ok")

1-element Vector{Tuple{Int64, String}}:
 (1, "ok")

In [165]:
# pure(1)
g = FunctionWrapper{Parser{Int},Tuple{String}}(x->pure(length(x)))
# bind(pure(""),g).p("")

FunctionWrapper{Parser{Int64}, Tuple{String}}(Ptr{Nothing} @0x00007eff985d90a0, Ptr{Nothing} @0x00007effa547c170, Base.RefValue{var"#335#336"}(var"#335#336"()), var"#335#336")